In [1]:
import pandas as pd

# 1.&nbsp; Import a csv file to DataFrame


In [11]:
url = "https://drive.google.com/file/d/1afxwDXfl-7cQ_qLwyDitfcCx3u7WMvkU/view?usp=share_link" # products.csv
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
products = pd.read_csv(path)

In [3]:
pd.options.display.max_rows = 10

### 1.1. &nbsp; Check .info() 
.shape() and .info() and .duplicated().sum() and .isna().sum()

In [4]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB


**INFO:**
- `sku` is not unique
- `desc` has 7 missing values
- `type` has 50 missing values, should be numerical datatype
- `price` has 46 missing values
- `price` should be numerical, needs to be explored
- ~`promo_price` should be numerical~  --> column has been deleted
- `in_stock` should be boolean
- `price` = 6.999.003 does not work
- `type` has scientific notation, value 1.44E+10, needs to be explored

# 2.&nbsp; Clean up missing & duplicates

## 2.1. Check for Missing Value

In [13]:
products["price"].isna().sum()

46

`price` has 46 missing values

In [7]:
# A closer look
products.loc[products['price'].isna(), :]

,sku,name,desc,price,promo_price,in_stock,type
34,TWS0019,Twelve South MagicWand support Apple Magic Tra...,MagicWand for wireless keyboard and Magic Trac...,NaN,299.899,0,8696
1900,AII0008,Aiino Case MacBook Air 11 '' Transparent,MacBook Air 11-inch casing with matte finish.,NaN,22.99,0,13835403
2039,CEL0020,Celly Ambo Luxury Leather Case + iPhone 6 Case...,Cover and housing together with magnet for iPh...,NaN,399.905,0,11865403
2042,CEL0007,Celly Wallet Case with removable cover Black i...,Case Book for iPhone 6 card case type.,NaN,128.998,0,11865403
2043,CEL0012,Celly Silicone Hard Shell iPhone 6 Blue,Hard Shell Silicone iPhone 6.,NaN,4.99,0,11865403
...,...,...,...,...,...,...,...
12098,CEL0050,Laser Case Celly iPhone 6 / 6S Plus Silver Clear,Protective Case for iPhone 6 Plus and 6s Plus,NaN,149.895,0,11865403
12100,CEL0037,Celly Frost Cover for iPhone 6 / 6S Black,Ultra Thin Case for iPhone 6 and 6s,NaN,99.898,0,11865403
12238,CEL0027,Celly Frost Cover for iPhone 6 / 6S White,Flexible plastic shell for iPhone 6 / 6S,NaN,89.903,0,11865403
12242,CEL0039,Celly Frost Cover for iPhone 6 / 6S Amarillo,Ultra Thin Case for iPhone 6 and 6s,NaN,89.903,0,11865403


In [8]:
#check the %
print(f"The missing values in price are {(products.price.isna().value_counts(normalize=True)[1] * 100).round(2)}% of all rows in the DataFrame")

The missing values in price are 0.24% of all rows in the DataFrame


As there is such a tiny amount of missing values, we will simply delete these rows, as we have enough data without them.

Option 1: `.loc`

In [16]:
products = products.loc[~products['price'].isna()]

Option 2: `.dropna()`

In [ ]:
# products = products.dropna(subset=['price'])

In [17]:
products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19280 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19280 non-null  object
 1   name         19280 non-null  object
 2   desc         19273 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19280 non-null  object
 5   in_stock     19280 non-null  int64 
 6   type         19230 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.2+ MB


In [14]:
products["desc"].isna().sum()

7

According to info `desc` has 7 missing values.

7 is a very small number to have missing, let's have a closer look

In [ ]:
products.loc[products['desc'].isna(), :]

We have 2 choices here:
* We can quickly and easily remove these rows.
* Or, alternatively, the products names here are quite descriptive, so I'm tempted to just copy them to the description column, so that there is a description if we later want utilise this column. I wouldn't recommend this if this DataFrame was the source of truth for our website. But this is not the case here, and we're not faking any information (guessing a price or so), so I'm happy with this option

In [ ]:
products.loc[products['desc'].isna(), 'desc'] = products.loc[products['desc'].isna(), 'name']

## 2.2 Check for Duplicates

In [19]:
# check for duplicates
products.duplicated().sum()

8746

In [20]:
# drop duplicates
products = products.drop_duplicates()

In [21]:
products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10534 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          10534 non-null  object
 1   name         10534 non-null  object
 2   desc         10527 non-null  object
 3   price        10534 non-null  object
 4   promo_price  10534 non-null  object
 5   in_stock     10534 non-null  int64 
 6   type         10484 non-null  object
dtypes: int64(1), object(6)
memory usage: 658.4+ KB


In [22]:
products.nunique()

sku            10534
name           10329
desc            7077
price           2690
promo_price     4613
in_stock           2
type             126
dtype: int64

In [23]:
products.shape

(10534, 7)

[DataFrame.size](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.size.html) returns the total number of values that the DataFrame has (the number of rows multiplied by the number of columns):

In [24]:
products.size

73738

We can check if the `.size` and `.shape` agree

In [25]:
products.shape[0] * products.shape[1] == products.size

True

# 3.&nbsp; Data types

We saw from looking at the output of .info() that both price and promo_price have been stored as objects and not as a numerical datatypes. Then we notice above that we have the dreaded two decimal point problem in both the `price` and `promo_price` columns? We can also see prices with 3 decimal places, prices should have 2 decimal places: we need to correct this before changing price as object to string

## 3.1 Decimal point problem in price


First, let's see how many values are affected by the 2 decimal point in price.

In [26]:
products.price.str.contains("\d+\.\d+\.\d+").value_counts()

False    10157
True       377
Name: price, dtype: int64

Two decimal point prices are invalid, we delete those rows.

In [27]:
products = products.loc[~products.price.astype(str).str.contains("\d+\.\d+\.\d+")]

In [28]:
# verify if above code is successfully executed, true are gone
products.price.str.contains("\d+\.\d+\.\d+").value_counts()

False    10157
Name: price, dtype: int64

Let's see how many values are affected by 3 decimal places in price.

In [29]:
products.price.str.contains("\d+\.\d{3,}").value_counts()

False    9992
True      165
Name: price, dtype: int64

In [30]:
# A closer look
products.loc[products['price'].str.contains("\d+\.\d{3,}")]

,sku,name,desc,price,promo_price,in_stock,type
362,REP0043,Speaker lower repair iPhone 4,Repair service including parts and labor for i...,499.004,499.004,0,"1,44E+11"
480,PIE0011,Internal Battery for iPhone 3G,Replacement AC Adapter for Apple iPhone 3G.,98.978,98.978,0,21485407
515,SEN0061,Sennheiser EZX 80 Handsfree iPhone iPad and iP...,IPhone bluetooth headset with microphone iPad ...,649.891,649.891,0,5384
518,SEV0026,Service installation RAM + HDD + SSD MacBook /...,RAM + HDD installation + SSD in your MacBook /...,599.918,599.918,0,20642062
525,SEV0024,Service installation RAM + HDD + SSD Mac mini,installation RAM HDD + SSD + on your Mac mini ...,599.918,599.918,0,20642062
...,...,...,...,...,...,...,...
19162,FIB0002-A,Open - Fibaro HomeKitt Sensor Door / Window,Sensor refurbished multi-functions for doors a...,599.918,501.067,0,11859
19185,RIN0017,Chime Bell Ring,Chime bell with free Wi-Fi to amplify your not...,350.005,350.005,0,NaN
19259,TPL0030-A,Open - TP-Link TL-PA4010P Passthrough Powerlin...,Refurbished Kit internet amplifiers with trans...,54.329,381.891,0,1334
19298,REP0428,iPhone Speaker lower repair 7,Repair service including parts and labor for i...,599.906,599.906,0,"1,44E+11"


Can't find any logic to manipulate the 3 decimal place price after cross checking with unit price in orderline.
So I will delete these too.

In [31]:
products = products.loc[~products.price.astype(str).str.contains("\d+\.\d{3,}")]

In [ ]:
# you can also delete both in one line

In [32]:
products = products.loc[(~products.price.astype(str).str.contains("\d+\.\d+\.\d+"))&(~products.price.astype(str).str.contains("\d+\.\d{3,}")), :]

In [33]:
# double check if delete works
products.price.str.contains("\d+\.\d{3,}").value_counts()

False    9992
Name: price, dtype: int64

In [ ]:
products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9992 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          9992 non-null   object
 1   name         9992 non-null   object
 2   desc         9992 non-null   object
 3   price        9992 non-null   object
 4   promo_price  9992 non-null   object
 5   in_stock     9992 non-null   int64 
 6   type         9946 non-null   object
dtypes: int64(1), object(6)
memory usage: 624.5+ KB


## 3.2 Change object to integer in price

In [ ]:
products['price'] = pd.to_numeric(products['price'])

<ipython-input-136-6416ab965591>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products['price'] = pd.to_numeric(products['price'])


In [ ]:
products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9992 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sku          9992 non-null   object 
 1   name         9992 non-null   object 
 2   desc         9992 non-null   object 
 3   price        9992 non-null   float64
 4   promo_price  9992 non-null   object 
 5   in_stock     9992 non-null   int64  
 6   type         9946 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 624.5+ KB


In [ ]:
products.isna().sum()

sku             0
name            0
desc            0
price           0
promo_price     0
in_stock        0
type           46
dtype: int64

leave type as it, not important for discount analysis

cross check price vs unit price

In [ ]:
products.loc[products['sku'] == "APP1214"] #APP1214 range from 89-63 in orderline

,sku,name,desc,price,promo_price,in_stock,type
10621,APP1214,Apple Magic Mouse 2,Bluetooth Apple Wireless Mouse,89.0,81.99,1,1387


we can assume price in products is regular price, promo price is error which doesn't make sense and also 92% of promo price is corrupted, see below:

## 3.3 Decimal point problem in promo price

Again, let's begin by seeing how many values are affected by the 2 problems, or the 3 decimal problem in promo price.

In [ ]:
promo_problems_number = products.loc[(products.promo_price.astype(str).str.contains("\d+\.\d+\.\d+"))|(products.promo_price.astype(str).str.contains("\d+\.\d{3,}")), :].shape[0]
promo_problems_number

9232

In [ ]:
print(f"The column promo_price has in total {promo_problems_number} wrong values. This is {round(((promo_problems_number / products.shape[0]) * 100), 2)}% of the rows of the DataFrame")

The column promo_price has in total 9232 wrong values. This is 92.39% of the rows of the DataFrame


In [ ]:
promo_price_df = products.loc[(products.promo_price.astype(str).str.contains("\d+\.\d+\.\d+"))|(products.promo_price.astype(str).str.contains("\d+\.\d{3,}")), :]
promo_price_df.sample(50)

,sku,name,desc,price,promo_price,in_stock,type
11062,PAC1293,Pack QNAP TS-251 + | 8GB RAM | Seagate 6TB,Pack QNAP TS-251 + with 8GB of RAM memory + 6T...,779.97,659.179,0,12175397
949,INC0232,Focal Incipio iPhone Case Camera Case SE / 5s ...,Housing for perfect photo to iPhone SE / 5s / ...,59.99,199.904,0,11865403
2958,LGE0017,LG Ultra Slim External DVD Writer White,superfine external recorder and portable DVD.,35.99,259.908,0,1424
18243,AP20379,Like new - Apple iPad 2 Wi-Fi Air 128GB Gold,Air iPad 2 Wi-Fi refitted color gold,539.00,4.590.002,0,42931714
17689,LGE0069,"LG 34UM59-P Monitor 34 ""sRGB 99% QHD HDMI",superwide 34-inch monitor image quality IPS sR...,399.00,3.599.895,1,1296
1102,IKM0033,IK Multimedia iRig Microphone Voice iPhone and...,color microphone for iPhone iPad and iPod Touch.,48.79,349.896,1,11905404
998,WDT0219,"Purple 4TB WD 35 ""Mac PC hard drive and NAS Se...",Western Digital hard disk video surveillance 4...,180.00,1.385.837,0,12655397
12222,WOE0012,Wowewa Protector Tempered Glass iPhone 6s Plus...,3D tempered glass protector: covers all edges ...,16.99,159.902,0,13555403
3089,SEA0092,"Seagate Expansion 2TB USB 3.0 Hard Disk 25 ""Black",25-inch External Hard Drive 2TB USB 3.0 for Ma...,104.99,929.946,1,11935397
13650,IHE0016-A,(Open) iHealth Wireless Pulse PO3M Oxómetro iP...,Oxómetro pulse with Bluetooth and APP for iPhone,79.95,631.111,0,1298


So we were correct, over 90% of the data in this column is corrupt. There's no point deleting all of these rows, then we would barely have a products table. Instead, as it's only this column that appears to be very untrustworthy, we will delete the column.

In [ ]:
products_cl = products.drop(["promo_price"], axis=1) #created a cleaned products file as products_cl

In [ ]:
#other option for writing the same code:
products_cl = products.drop(columns=["promo_price"])

In [ ]:
# To apply directly to the original dataframe, we need to indicate inplace = True.
products.drop(["promo_price"], axis=1, inplace = True)

## 3.4 Take a look at the expensive and cheap products

In [ ]:
products_cl.nlargest(5, "price")

,sku,name,desc,price,in_stock,type
18429,APP2660,"Apple iMac Pro 27 ""18-core Intel Xeon W 23GHz ...",Pro iMac 27 inch screen Retina 5K and Intel Xe...,15339.0,0,118692158
18465,APP2696,"Apple iMac Pro 27 ""18-core Intel Xeon W 23GHz ...",Pro iMac 27 inch screen Retina 5K and Intel Xe...,14619.0,0,118692158
18428,APP2659,"Apple iMac Pro 27 ""14-core Intel Xeon W 25GHz ...",Pro iMac 27 inch screen Retina 5K and Intel Xe...,14379.0,0,118692158
18464,APP2695,"Apple iMac Pro 27 ""14-core Intel Xeon W 25GHz ...",Pro iMac 27 inch screen Retina 5K and Intel Xe...,13659.0,0,118692158
18425,APP2656,"Apple iMac Pro 27 ""18-core Intel Xeon W 23GHz ...",Pro iMac 27 inch screen Retina 5K and Intel Xe...,13419.0,0,118692158


In [ ]:
products_cl.nsmallest(5, "price")

,sku,name,desc,price,in_stock,type
1310,PAR0037,Upper body MiniDrone Parrot Jumping Sumo White,MiniDrone upper body Sumo Jumping.,2.99,0,11905404
2362,IFX0017,iFixit SIM Card Eject Tool for iPhone and iPad,Tool for easy removal of the SIM card for iPho...,2.99,1,12645406
18821,DOD0015,Dodocool Antideslizantes Replacement Cases for...,slip silicone case that enhance comfort and st...,3.49,0,5384
2829,LEX0009,Memory Card Lexar microSDHC Class 10 | 8GB wit...,Memory Card 8GB microSDHC Class 10 speed 45MB ...,3.79,0,57445397
35,BEL0072,Belkin PRO Series USB 2.0 Cable 18M,USB Cable USB type A and type B 18m in length ...,3.99,1,1325


Don't forget to download/save your new DataFrames. Also, give them an obvious name, so that you know they are the cleaned version and not the original DataFrame.

In [ ]:
from google.colab import files
products_cl.to_csv("products_cl.csv", index=False)
files.download("products_cl.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>